# Hands-on - 1. Data Discovery using Azure REST API

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 - AzureML<br>
__Required Packages:__ No<br>
__Platforms Supported:__  Azure Machine Learning Notebooks
     
__Data Source Required:__ Log Analytics tables 
    
### Description
This notebook will provide step-by-step instructions and sample code to guide you through Azure authentication, Sentinel data discovery by using Azure REST API.<br>
*** No need to download and install any other Python modules. ***<br>
*** Please run the cells sequentially to avoid errors. *** <br>

## Table of Contents
1. Warm-up
2. Azure Authentication
3. List Sentinel Watchlists Using API
4. List Sentinel Incidents Using API

## 1. Warm-up

In [ ]:
# If you need to know what Python modules are available, you may run this:
# help("modules")

In [ ]:
# Loading Python libraries
from azure.common.credentials import get_azure_cli_credentials
import requests
import json
import pandas

In [ ]:
# Functions will be used in this notebook
def read_config_values(file_path):
    "This loads pre-generated parameters for Sentinel Workspace"
    with open(file_path) as json_file:
        if json_file:
            json_config = json.load(json_file)
            return (json_config["tenant_id"],
                    json_config["subscription_id"],
                    json_config["resource_group"],
                    json_config["workspace_id"],
                    json_config["workspace_name"])
    return None

# Calling Sentinel API for List, the same template can be used for calling other Azure REST APIs with different parameters.
# For different environments, such as national clouds, you may need to use different root_url, please contact with your admins.
# It can be ---.azure.us, ---.azure.microsoft.scloud, ---.azure.eaglex.ic.gov, etc.
def call_azure_rest_api_for_list(token, resource_name, api_version):
    "Calling Sentinel REST API"
    headers = {"Authorization": token, "content-type":"application/json" }
    provider_name = "Microsoft.OperationalInsights"
    provider2_name = "Microsoft.SecurityInsights"
    target_resource_name = resource_name
    api_version = api_version
    root_url = "https://management.azure.com"
    arm_rest_url_template_for_list = "{0}/subscriptions/{1}/resourceGroups/{2}/providers/{3}/workspaces/{4}/providers/{5}/{6}?api-version={7}"
    arm_rest_url = arm_rest_url_template_for_list.format(root_url, subscription_id, resource_group, provider_name, workspace_name, provider2_name, target_resource_name, api_version)
    response = requests.get(arm_rest_url, headers=headers, verify=True)
    return response

def display_result_name(response):
    "Default to display column - name, you may change it to other columns"
    column_name = "name"
    if response != None:
        entries = [item[column_name] for item in response.json()["value"]] 
        display(entries)

def display_result(response):
    "Display the result set as pandas.DataFrame"
    if response != None:
        df = pandas.DataFrame(response_incident.json()["value"])
        display(df)

In [ ]:
# Calling the above function to populate Sentinel workspace parameters
# The file, config.json, was generated by the system, however, you may modify the values, or manually set the variables
tenant_id, subscription_id, resource_group, workspace_id, workspace_name = read_config_values('config.json');

## 2. Azure Authentication

In [ ]:
# Azure CLI is used to get device code to login into Azure, you need to copy the code and open the DeviceLogin site.
# You may add [--tenant $tenant_id] to the command
!az login --use-device-code --subscription_id subscription_id

In [ ]:
# Extract access token, which will be used to access Sentinel Watchlist API for your Watchlist data.  
credentials, sub_id = get_azure_cli_credentials()
creds = credentials._get_cred(resource=None)
token = creds._token_retriever()[2]
access_token = token['accessToken']
header_token_value = "Bearer {}".format(access_token)

## 3. List Sentinel Watchlists Using API

In [ ]:
# Calling Sentinel Watchlist API
# If you don't have Watchlist, you may create one, or try to access different features, such as Bookmarks.
response_watchlist = call_azure_rest_api_for_list(header_token_value, "watchlists", "2019-01-01-preview")

In [ ]:
# Display the result
display_result_name(response_watchlist)

## 4. List Sentinel Incidents Using API

In [ ]:
# Calling Sentinel Incident API
# If you don't have incidents, you may create one through Azure Portal.
response_incident = call_azure_rest_api_for_list(header_token_value, "incidents", "2020-01-01")

In [ ]:
# Display the result
display_result(response_incident)

Thanks for coming along all the way to the end.  In the next Hands-on notebook, I will show you how to access data using Azure SDK for Python.  And keep one of the watchlist name, it will be used in the next notebook.  A la prochaine.